In [1]:
#Set environment options
import pandas as pd
# settings to display all columns
pd.set_option("display.max_columns", None)

### Set environmental variables and install libraries

# Call nba_api for all NBA games in a given season

In [2]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
import time

#get nba team data including id's
nba_teams = pd.DataFrame(teams.get_teams())
#Create list of NBA team id's
team_ids = nba_teams['id'].tolist()

#season data should be pulled for 
s = '2022-23'


#function to call api and get all NBA games for a given season
def get_all_games(team__ids,season):
    team_season = []
    for i in team_ids:
        games = leaguegamefinder.LeagueGameFinder(team_id_nullable = i,season_nullable = season,season_type_nullable ='Regular Season').get_data_frames()[0]
        team_season.append(games)
        nba_season = pd.concat(team_season)
        time.sleep(1)
    return(nba_season)

nba_season = get_all_games(team_ids,s)

In [3]:
nba_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 12
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          350 non-null    object 
 1   TEAM_ID            350 non-null    int64  
 2   TEAM_ABBREVIATION  350 non-null    object 
 3   TEAM_NAME          350 non-null    object 
 4   GAME_ID            350 non-null    object 
 5   GAME_DATE          350 non-null    object 
 6   MATCHUP            350 non-null    object 
 7   WL                 343 non-null    object 
 8   MIN                350 non-null    int64  
 9   PTS                350 non-null    int64  
 10  FGM                350 non-null    int64  
 11  FGA                350 non-null    int64  
 12  FG_PCT             350 non-null    float64
 13  FG3M               350 non-null    int64  
 14  FG3A               350 non-null    int64  
 15  FG3_PCT            350 non-null    float64
 16  FTM                350 non-

Get team stats for specific team

In [14]:
# subset TEAM_ABBREVIATION = 'DAL'
NBA_seasons_DAL = nba_season[nba_season.MATCHUP.str.contains('DAL',regex=False)]

In [4]:
#function to combine matchup data
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        #pass
        return None
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result
    

In [13]:
# Combine the game rows into one. By default, the home team will be TEAM_A.
game_df = combine_team_games(nba_season,keep_method='home')
game_df = game_df.sort_values(by= 'GAME_DATE')
game_df['TOTAL_GAME_PTS'] = game_df['PTS_A'] + game_df['PTS_B']
game_df.head(20)

,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,FGM_A,FGA_A,FG_PCT_A,FG3M_A,FG3A_A,FG3_PCT_A,FTM_A,FTA_A,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLUS_MINUS_A,TEAM_ID_B,TEAM_ABBREVIATION_B,TEAM_NAME_B,MATCHUP_B,WL_B,MIN_B,PTS_B,FGM_B,FGA_B,FG_PCT_B,FG3M_B,FG3A_B,FG3_PCT_B,FTM_B,FTA_B,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B,TOTAL_GAME_PTS
109,22022,1610612738,BOS,Boston Celtics,0022200001,2022-10-18,BOS vs. PHI,W,240,126,46,82,0.561,12,35,0.343,22,28,0.786,6,30,36,24,8,3,10,24,9.0,1610612755,PHI,Philadelphia 76ers,PHI @ BOS,L,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,-9.0,243
377,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,45,99,0.455,16,45,0.356,17,23,0.739,11,37,48,31,11,4,18,23,14.0,1610612747,LAL,Los Angeles Lakers,LAL @ GSW,L,241,109,40,94,0.426,10,40,0.250,19,25,0.760,9,39,48,23,12,4,21,18,-14.0,232
601,22022,1610612750,MIN,Minnesota Timberwolves,0022200010,2022-10-19,MIN vs. OKC,W,241,115,39,94,0.415,10,38,0.263,27,33,0.818,14,41,55,25,6,5,14,21,7.0,1610612760,OKC,Oklahoma City Thunder,OKC @ MIN,L,241,108,38,99,0.384,14,45,0.311,18,22,0.818,16,41,57,22,6,6,13,26,-7.0,223
658,22022,1610612763,MEM,Memphis Grizzlies,0022200009,2022-10-19,MEM vs. NYK,W,266,115,42,108,0.389,17,50,0.340,14,23,0.609,20,44,64,27,6,7,13,17,3.0,1610612752,NYK,New York Knicks,NYK @ MEM,L,265,112,44,109,0.404,9,37,0.243,15,18,0.833,18,36,54,28,7,5,12,26,-3.0,227
53,22022,1610612737,ATL,Atlanta Hawks,0022200005,2022-10-19,ATL vs. HOU,W,240,117,45,90,0.500,7,25,0.280,20,24,0.833,4,34,38,30,12,5,9,18,10.0,1610612745,HOU,Houston Rockets,HOU @ ATL,L,240,107,42,98,0.429,9,35,0.257,14,15,0.933,15,39,54,25,4,3,15,20,-10.0,224
154,22022,1610612761,TOR,Toronto Raptors,0022200008,2022-10-19,TOR vs. CLE,W,240,108,36,86,0.419,13,30,0.433,23,32,0.719,12,29,41,19,15,4,13,23,3.0,1610612739,CLE,Cleveland Cavaliers,CLE @ TOR,L,242,105,38,77,0.494,10,28,0.357,19,23,0.826,6,32,38,25,5,5,17,21,-3.0,213
246,22022,1610612748,MIA,Miami Heat,0022200007,2022-10-19,MIA vs. CHI,L,240,108,37,80,0.463,12,28,0.429,22,28,0.786,8,38,46,21,6,2,19,24,-8.0,1610612741,CHI,Chicago Bulls,CHI @ MIA,W,240,116,41,93,0.441,13,36,0.361,21,28,0.750,12,34,46,23,12,6,14,26,8.0,224
334,22022,1610612762,UTA,Utah Jazz,0022200012,2022-10-19,UTA vs. DEN,W,242,123,42,83,0.506,16,38,0.421,23,31,0.742,11,32,43,27,10,1,19,21,21.0,1610612743,DEN,Denver Nuggets,DEN @ UTA,L,238,102,40,83,0.482,5,22,0.227,17,18,0.944,10,25,35,21,10,3,21,23,-21.0,225
705,22022,1610612754,IND,Indiana Pacers,0022200004,2022-10-19,IND vs. WAS,L,241,107,39,97,0.402,15,42,0.357,14,21,0.667,12,30,42,21,7,5,15,20,-7.0,1610612764,WAS,Washington Wizards,WAS @ IND,W,241,114,42,92,0.457,11,31,0.355,19,24,0.792,14,39,53,21,5,10,16,19,7.0,221
206,22022,1610612751,BKN,Brooklyn Nets,0022200006,2022-10-19,BKN vs. NOP,L,242,108,40,89,0.449,10,33,0.303,18,24,0.750,9,30,39,22,10,10,16,25,-22.0,1610612740,NOP,New Orleans Pelicans,NOP @ BKN,W,240,130,50,102,0.490,12,26,0.462,18,25,0.720,21,40,61,31,9,4,12,21,22.0,238


# Player Rotation by game api

In [10]:
#home and away player rotations by game
from nba_api.stats.endpoints import gamerotation

gr = gamerotation.GameRotation(game_id= '0022200149')

gamerotation_away = gr.get_data_frames()[0]
gamerotation_home = gr.get_data_frames()[1]

# BoxScoreSummary
* game_summary:  
['GAME_DATE_EST', 'GAME_SEQUENCE', 'GAME_ID', 'GAME_STATUS_ID', 'GAME_STATUS_TEXT', 'GAMECODE', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON', 'LIVE_PERIOD', 'LIVE_PC_TIME', 'NATL_TV_BROADCASTER_ABBREVIATION', 'LIVE_PERIOD_TIME_BCAST', 'WH_STATUS']

* other_stats:  
['LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PTS_PAINT', 'PTS_2ND_CHANCE', 'PTS_FB', 'LARGEST_LEAD', 'LEAD_CHANGES', 'TIMES_TIED', 'TEAM_TURNOVERS', 'TOTAL_TURNOVERS', 'TEAM_REBOUNDS', 'PTS_OFF_TO']

* officials:  
['OFFICIAL_ID', 'FIRST_NAME', 'LAST_NAME', 'JERSEY_NUM']

* inactive_players:  
['PLAYER_ID', 'FIRST_NAME', 'LAST_NAME', 'JERSEY_NUM', 'TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'TEAM_ABBREVIATION']

* game_info:  
['GAME_DATE', 'ATTENDANCE', 'GAME_TIME']

* line_score:  
['GAME_DATE_EST', 'GAME_SEQUENCE', 'GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY_NAME', 'TEAM_NICKNAME', 'TEAM_WINS_LOSSES', 'PTS_QTR1', 'PTS_QTR2', 'PTS_QTR3', 'PTS_QTR4', 'PTS_OT1', 'PTS_OT2', 'PTS_OT3', 'PTS_OT4', 'PTS_OT5', 'PTS_OT6', 'PTS_OT7', 'PTS_OT8', 'PTS_OT9', 'PTS_OT10', 'PTS']

* last_meeting:  
['GAME_ID', 'LAST_GAME_ID', 'LAST_GAME_DATE_EST', 'LAST_GAME_HOME_TEAM_ID', 'LAST_GAME_HOME_TEAM_CITY', 'LAST_GAME_HOME_TEAM_NAME', 'LAST_GAME_HOME_TEAM_ABBREVIATION', 'LAST_GAME_HOME_TEAM_POINTS', 'LAST_GAME_VISITOR_TEAM_ID', 'LAST_GAME_VISITOR_TEAM_CITY', 'LAST_GAME_VISITOR_TEAM_NAME', 'LAST_GAME_VISITOR_TEAM_CITY1', 'LAST_GAME_VISITOR_TEAM_POINTS']

* season_series:  
['GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'GAME_DATE_EST', 'HOME_TEAM_WINS', 'HOME_TEAM_LOSSES', 'SERIES_LEADER']

* available_video:  
['GAME_ID', 'VIDEO_AVAILABLE_FLAG', 'PT_AVAILABLE', 'PT_XYZ_AVAILABLE', 'WH_STATUS', 'HUSTLE_STATUS', 'HISTORICAL_STATUS']

In [27]:
from nba_api.stats.endpoints import boxscoresummaryv2

bxv2_call = boxscoresummaryv2.BoxScoreSummaryV2(game_id='0022200149')

game_summary = bxv2_call.get_data_frames()[0]
other_stats = bxv2_call.get_data_frames()[1]
officials = bxv2_call.get_data_frames()[2]
inactive_players = bxv2_call.get_data_frames()[3]
game_info = bxv2_call.get_data_frames()[4]
line_score = bxv2_call.get_data_frames()[5]
last_meeting = bxv2_call.get_data_frames()[6]
season_series = bxv2_call.get_data_frames()[7]
available_video = bxv2_call.get_data_frames()[8]


# BoxScoreAdvancedV2

* player_stats:  
['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID', 'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE']

* team_stats:  
['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'E_TM_TOV_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE']

In [31]:
from nba_api.stats.endpoints import boxscoreadvancedv2

call = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id='0022200149')
player_stats = call.get_data_frames()[0]
team_stats = call.get_data_frames()[1]


# CumeStatsTeam

* game_by_game_stats:  
['JERSEY_NUM', 'PLAYER', 'PERSON_ID', 'TEAM_ID', 'GP', 'GS', 'ACTUAL_MINUTES', 'ACTUAL_SECONDS', 'FG', 'FGA', 'FG_PCT', 'FG3', 'FG3A', 'FG3_PCT', 'FT', 'FTA', 'FT_PCT', 'OFF_REB', 'DEF_REB', 'TOT_REB', 'AST', 'PF', 'DQ', 'STL', 'TURNOVERS', 'BLK', 'PTS', 'MAX_ACTUAL_MINUTES', 'MAX_ACTUAL_SECONDS', 'MAX_REB', 'MAX_AST', 'MAX_STL', 'MAX_TURNOVERS', 'MAX_BLKP', 'MAX_PTS', 'AVG_ACTUAL_MINUTES', 'AVG_ACTUAL_SECONDS', 'AVG_REB', 'AVG_AST', 'AVG_STL', 'AVG_TURNOVERS', 'AVG_BLKP', 'AVG_PTS', 'PER_MIN_REB', 'PER_MIN_AST', 'PER_MIN_STL', 'PER_MIN_TURNOVERS', 'PER_MIN_BLK', 'PER_MIN_PTS']

* total_team_stats:  
['CITY', 'NICKNAME', 'TEAM_ID', 'W', 'L', 'W_HOME', 'L_HOME', 'W_ROAD', 'L_ROAD', 'TEAM_TURNOVERS', 'TEAM_REBOUNDS', 'GP', 'GS', 'ACTUAL_MINUTES', 'ACTUAL_SECONDS', 'FG', 'FGA', 'FG_PCT', 'FG3', 'FG3A', 'FG3_PCT', 'FT', 'FTA', 'FT_PCT', 'OFF_REB', 'DEF_REB', 'TOT_REB', 'AST', 'PF', 'STL', 'TOTAL_TURNOVERS', 'BLK', 'PTS', 'AVG_REB', 'AVG_PTS', 'DQ']

In [43]:
from nba_api.stats.endpoints import cumestatsteam

call = cumestatsteam.CumeStatsTeam(game_ids='0022200149',team_id='1610612749')

game_by_game_stats = call.get_data_frames()[0]
test = call.get_data_frames()[1]
test.head()

,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,TEAM_REBOUNDS,GP,GS,ACTUAL_MINUTES,ACTUAL_SECONDS,FG,FGA,FG_PCT,FG3,FG3A,FG3_PCT,FT,FTA,FT_PCT,OFF_REB,DEF_REB,TOT_REB,AST,PF,STL,TOTAL_TURNOVERS,BLK,PTS,AVG_REB,AVG_PTS,DQ
0,Milwaukee,Bucks,1610612749,0,1,0,0,0,1,1,12,1,5,240,0,36,87,0.414,13,40,0.325,13,16,0.813,6,35,53,21,18,5,19,9,98,53.0,98.0,0
1,OPPONENTS,,0,1,0,1,0,0,0,0,9,1,5,240,0,46,99,0.465,11,29,0.379,14,17,0.824,12,36,57,23,18,11,12,3,117,57.0,117.0,0


# CumeStatsTeamGames
Get a list of game_id for a team in a given season

* cume_stats_team_games: ['MATCHUP', 'GAME_ID']


In [51]:
from nba_api.stats.endpoints import cumestatsteamgames

call = cumestatsteamgames.CumeStatsTeamGames(team_id='1610612749',season=s)
cume_stats_team_games = call.get_data_frames()[0]
cume_stats_team_games.head()


,MATCHUP,GAME_ID
0,11/14/2022 Hawks at Bucks,0022200204
1,11/11/2022 Bucks at Spurs,0022200180
2,11/09/2022 Bucks at Thunder,0022200168
3,11/07/2022 Bucks at Hawks,0022200149
4,11/05/2022 Thunder at Bucks,0022200136


In [14]:
from nba_api.stats.endpoints import matchupsrollup

call =  matchupsrollup.MatchupsRollup(season='2022-23')
matchupsrollup_df = call.get_data_frames()[0]